#Импорт библиотек

In [1]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
import numpy as np
import pymorphy2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#Стоп слова

In [3]:
def get_list_stop():
  stop_words = list(get_stop_words('russian'))
  print(f'Из библиотеки stop_words - {len(stop_words)} слов')
  stop_words_nltk = list(stopwords.words('russian'))
  print(f'Из nltk - {len(stop_words_nltk)} слов')
  stop_words.extend(stop_words_nltk)
  print(f'Всего - {len(stop_words)} слов')
  return stop_words

In [4]:
stop_words = get_list_stop()
print(f'Примеры стоп слов {stop_words[50:70]} ...')

Из библиотеки stop_words - 421 слов
Из nltk - 151 слов
Всего - 572 слов
Примеры стоп слов ['ваш', 'вон', 'вот', 'все', 'всю', 'вся', 'всё', 'где', 'год', 'два', 'две', 'дел', 'для', 'его', 'ему', 'еще', 'ещё', 'или', 'ими', 'имя'] ...


#Работа с текстом

Корпус документов

In [5]:
fname = 'corp0.txt'

Лемматизатор с русского языка

In [6]:
morph = pymorphy2.MorphAnalyzer()

Обработка документов из файла

In [7]:
def text_processing(fname, stop_words, morph):
  with open(fname, 'r', encoding = "utf_8", errors="ignore") as read_txt:
    text = read_txt.read()
    text = re.sub(r'[^А-я #\n]', '', text)
    text = text.lower()
    text = text.split('\n')
    for i in range(len(text)):
      text[i] = nltk.word_tokenize(text[i])
      text[i] = [morph.parse(word)[0].normal_form for word in text[i] if word not in stop_words]
  return text

In [8]:
list_doc = text_processing(fname, stop_words, morph)
list_doc[0]

['#',
 'семья',
 'ребёнок',
 'хотеть',
 'вылезать',
 'река',
 'каждый',
 'день',
 'допустимый',
 'просмотр',
 'телевизор',
 'становиться',
 'уверить',
 'способный',
 'вынести',
 'крик',
 'половина',
 'мой']

In [9]:
x, y = [], [] # документы, их метки
for i in range(len(list_doc)):
    y.append(list_doc[i][1])
    x.append(list_doc[i][2:])
x, y = np.array(x), np.array(y)

<ipython-input-9-bff1fd503610>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x, y = np.array(x), np.array(y)


In [10]:
x[0]

['ребёнок',
 'хотеть',
 'вылезать',
 'река',
 'каждый',
 'день',
 'допустимый',
 'просмотр',
 'телевизор',
 'становиться',
 'уверить',
 'способный',
 'вынести',
 'крик',
 'половина',
 'мой']

In [11]:
y

array(['семья', 'семья', 'семья', ..., 'наука', 'политика', 'страна'],
      dtype='<U12')

In [12]:
for i in range(len(x)):
    x[i] = ' '.join(x[i])

In [13]:
x[0]

'ребёнок хотеть вылезать река каждый день допустимый просмотр телевизор становиться уверить способный вынести крик половина мой'

#Векторизация данных с использованием CountVectorizer

In [14]:
k_split = 0.25
x_trn, x_vl, y_trn, y_vl = train_test_split(x, y, test_size = k_split, shuffle = True, stratify=y)

In [15]:
vec = CountVectorizer(token_pattern = '\w+', binary = False)
len_trn = len(x_trn)
x_trn = np.append(x_trn, x_vl, axis = 0) # Объединяем x_trn и x_vl и получаем полный корпус
x_trn = vec.fit_transform(x_trn)
x_vl = x_trn[len_trn:]
x_trn = x_trn[:len_trn]
# Преобразуем разреженные матрицы в массивы
x_trn = np.float32(x_trn.toarray())
x_vl = np.float32(x_vl.toarray())

In [16]:
x_trn[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [17]:
x_trn.shape[1]

34989

Перечень классов документов

In [18]:
classes = dict.fromkeys(np.unique(y), 0)
for i in range(len(list_doc)):
  classes[list_doc[i][1]] += 1

In [19]:
classes

{'автомобиль': 249,
 'здоровье': 157,
 'культура': 358,
 'наука': 227,
 'недвижимость': 98,
 'политика': 600,
 'происшествие': 436,
 'реклама': 94,
 'семья': 101,
 'спорт': 373,
 'страна': 146,
 'техника': 289,
 'экономика': 272}

#Применение выбранных классификаторов

__KNeighbors__ – объект присваивается тому классу, который является наиболее распространённым среди k соседей данного элемента, классы которых уже известны

__PassiveAggressiveClassifier__ – алгоритм реагирует агрессивно (меняет свои веса) на неверно классифицированные примеры и остается пассивным (не изменяет веса) в случае правильной классификации

__LinearDiscriminantAnalysis__ – строит линейные комбинации признаков для максимизации разделения между классами данных. Алгоритм использует информацию о разбросе между классами и внутри классов для определения оптимальной гиперплоскости, разделяющей классы в признаковом пространстве

In [21]:
def train(classifier):
    start = time.time()
    classifier.fit(x_trn, y_trn)
    print('Время обучения классификатора:', time.time() - start)
    print('Оценка точности классификации')
    train_score = classifier.score(x_trn, y_trn)
    print('Точность на обучающем множестве:', round(train_score, 4))
    val_score = classifier.score(x_vl, y_vl)
    print('Точность на проверочном множестве:', round(val_score, 4))
    return classifier, train_score, val_score

In [22]:
print('KNeighbors')
knn = KNeighborsClassifier(n_neighbors=12)
knn, knn_train, knn_val = train(knn)
print('---')
print('LDA')
lda = LinearDiscriminantAnalysis()
lda, lda_train, lda_val = train(lda)
print('---')
print('PassiveAggressive')
pa = PassiveAggressiveClassifier()
pa, pa_train, pa_val = train(pa)

KNeighbors
Время обучения классификатора: 0.044127702713012695
Оценка точности классификации
Точность на обучающем множестве: 0.5286
Точность на проверочном множестве: 0.4529
---
LDA
Время обучения классификатора: 110.69402885437012
Оценка точности классификации
Точность на обучающем множестве: 0.9773
Точность на проверочном множестве: 0.3447
---
PassiveAggressive
Время обучения классификатора: 40.55483412742615
Оценка точности классификации
Точность на обучающем множестве: 0.9996
Точность на проверочном множестве: 0.8612


Оценка точности на каждом классе

In [24]:
uniq = classes.keys()

In [25]:
print('KNeighbors')
print('Train')
print(classification_report(y_trn, knn.predict(x_trn),
          target_names = uniq, digits = 4))
print('Val')
print(classification_report(y_vl, knn.predict(x_vl),
          target_names = uniq, digits = 4))

KNeighbors
Train
              precision    recall  f1-score   support

  автомобиль     0.9118    0.3316    0.4863       187
    здоровье     0.5455    0.0508    0.0930       118
    культура     0.8344    0.4888    0.6165       268
       наука     0.9429    0.5824    0.7200       170
недвижимость     1.0000    0.2192    0.3596        73
    политика     0.9077    0.6778    0.7761       450
происшествие     0.8982    0.6208    0.7342       327
     реклама     0.8000    0.0563    0.1053        71
       семья     0.4412    0.1974    0.2727        76
       спорт     0.9014    0.6857    0.7789       280
      страна     0.8000    0.1468    0.2481       109
     техника     0.1667    0.9585    0.2840       217
   экономика     0.8198    0.4461    0.5778       204

    accuracy                         0.5286      2550
   macro avg     0.7669    0.4202    0.4656      2550
weighted avg     0.7950    0.5286    0.5717      2550

Val
              precision    recall  f1-score   support

  а

In [26]:
print('LDA')
print('Train')
print(classification_report(y_trn, lda.predict(x_trn),
          target_names = uniq, digits = 4))
print('Val')
print(classification_report(y_vl, lda.predict(x_vl),
          target_names = uniq, digits = 4))

LDA
Train
              precision    recall  f1-score   support

  автомобиль     1.0000    0.9893    0.9946       187
    здоровье     1.0000    0.9831    0.9915       118
    культура     0.9158    0.9739    0.9439       268
       наука     0.9011    0.9647    0.9318       170
недвижимость     1.0000    0.9178    0.9571        73
    политика     1.0000    0.9933    0.9967       450
происшествие     0.9969    0.9817    0.9892       327
     реклама     1.0000    0.9859    0.9929        71
       семья     0.9859    0.9211    0.9524        76
       спорт     0.9786    0.9821    0.9804       280
      страна     0.9464    0.9725    0.9593       109
     техника     0.9953    0.9724    0.9837       217
   экономика     0.9950    0.9755    0.9851       204

    accuracy                         0.9773      2550
   macro avg     0.9781    0.9702    0.9737      2550
weighted avg     0.9783    0.9773    0.9775      2550

Val
              precision    recall  f1-score   support

  автомоби

In [27]:
print('PassiveAggressive')
print('Train')
print(classification_report(y_trn, pa.predict(x_trn),
          target_names = uniq, digits = 4))
print('Val')
print(classification_report(y_vl, pa.predict(x_vl),
          target_names = uniq, digits = 4))

PassiveAggressive
Train
              precision    recall  f1-score   support

  автомобиль     1.0000    1.0000    1.0000       187
    здоровье     1.0000    1.0000    1.0000       118
    культура     1.0000    1.0000    1.0000       268
       наука     1.0000    1.0000    1.0000       170
недвижимость     1.0000    1.0000    1.0000        73
    политика     0.9978    1.0000    0.9989       450
происшествие     1.0000    1.0000    1.0000       327
     реклама     1.0000    1.0000    1.0000        71
       семья     1.0000    1.0000    1.0000        76
       спорт     1.0000    1.0000    1.0000       280
      страна     1.0000    0.9908    0.9954       109
     техника     1.0000    1.0000    1.0000       217
   экономика     1.0000    1.0000    1.0000       204

    accuracy                         0.9996      2550
   macro avg     0.9998    0.9993    0.9996      2550
weighted avg     0.9996    0.9996    0.9996      2550

Val
              precision    recall  f1-score   suppo

Классификатор __KNN__ оказался наиболее быстр, но наименее точен.

Классификатор __LDA__ обучался дольше, при этом случилось переобучение.

Классификтор __PA__ оказался наиболее медленным, при этом на тесте показал высокую точность.